This is an attempt at sketching the generalized equilibrium solver with N-D mapping.

In [2]:
%%timeit
import numpy as np
from itertools import chain

x1 = np.random.RandomState(42).rand(100000, 3)
x1 /= x1.sum(axis=1)[:, np.newaxis]
x1 = np.concatenate((np.array([[1e-16, 1e-16, 1], [1e-16, 1, 1e-16], [1, 1e-16, 1e-16]]), x1), axis=0)
energies = -8.3145*1000*np.sum(x1*np.log(x1), axis=-1)
energies[[0, 1, 2]] = [100000, 100000, 100000]
dof_values = np.array([[1/3, 1/3, 1/3], [0.2, 0.3, 0.5], [0.3, 0.2, 0.5], [0.5, 0.3, 0.2]])
dof_energies = np.array([100000., 100000., 100000., 100000.])
dof_simplices = np.empty(dof_values.shape, dtype=np.int)
dof_simplices[..., :] = np.arange(x1.shape[-1])
dof_potentials = np.empty(dof_values.shape, dtype=np.float)
dof_potentials[...] = np.inf
trial_points = np.empty(tuple(chain(dof_values.shape[0:-1])), dtype=np.int)
# Initialize trial point as lowest energy point in the system
# Trial simplices will be the current simplex with each vertex replaced by the trial point
trial_points[...] = np.argmin(energies, axis=-1)
trial_simplices = np.empty(tuple(chain(dof_values.shape, [x1.shape[-1]])), dtype=np.int)
trial_simplices[..., :, :] = np.tile(np.arange(x1.shape[-1]), (x1.shape[-1], 1))
trial_simplices[..., np.arange(x1.shape[-1]), np.arange(x1.shape[-1])] = trial_points[..., np.newaxis]
#trial_simplices[..., 0, :] = 0
trial_matrix = np.swapaxes(x1[trial_simplices], -1, -2)
nondegenerate_indices = np.all(np.linalg.svd(trial_matrix, compute_uv=False) > 1e-6, axis=-1)
nondegenerate_simplices = trial_simplices[nondegenerate_indices]
dof_sum_array = np.sum(nondegenerate_indices, axis=-1, dtype=np.int)
dof_index_array = np.repeat(np.arange(dof_values.shape[-2], dtype=np.int), dof_sum_array.astype(np.int))
fractions = np.linalg.solve(trial_matrix[nondegenerate_indices], dof_values[dof_index_array])
bounding_simplices = np.all(fractions > 0, axis=-1)
candidate_simplices = nondegenerate_simplices[bounding_simplices]
candidate_potentials = np.linalg.solve(x1[candidate_simplices],
                                       energies[candidate_simplices])

dof_index_array = np.repeat(np.arange(dof_values.shape[-2], dtype=np.int), trial_simplices.shape[-2])

target_values = dof_values[dof_index_array[bounding_simplices]]
candidate_energies = np.tensordot(candidate_potentials, target_values, axes=(-1, -1))
comparison_matrix = np.where(dof_index_array[bounding_simplices] == np.arange(dof_values.shape[-2])[..., np.newaxis],
                             candidate_energies, np.inf)
lowest_candidate_indices = np.argmin(comparison_matrix, axis=-1)
#print((candidate_energies[..., np.arange(dof_values.shape[-2]), lowest_candidate_indices] < dof_energies))
dof_simplices = np.where((candidate_energies[..., np.arange(dof_values.shape[-2]),
                                             lowest_candidate_indices] < dof_energies)[..., np.newaxis],
                         candidate_simplices[lowest_candidate_indices], dof_simplices)
#print(dof_simplices)
dof_potentials = np.where((candidate_energies[..., np.arange(dof_values.shape[-2]),
                                             lowest_candidate_indices] < dof_energies)[..., np.newaxis],
                          candidate_potentials[lowest_candidate_indices], dof_potentials)
#print(dof_simplices)
#print(dof_potentials)
dof_energies = np.minimum(dof_energies, candidate_energies[..., np.arange(dof_values.shape[-2]),
                                                           lowest_candidate_indices])
driving_forces = np.tensordot(dof_potentials, x1, axes=(-1, -1)) - energies
trial_points = np.argmax(driving_forces, axis=-1)
found_equilibrium = np.all(driving_forces < 1e-4 * 8.3145 * 10000)
#print(driving_forces)
#print(trial_points)
#print(found_equilibrium)
#print(dof_energies)

10 loops, best of 3: 54.1 ms per loop
